In [1]:
import re
import pandas as pd
import numpy as np

from transform import pick_date, filter_date, substract_months, maintain_alpha, remove_html, remove_morespace, remove_single, repair_salary

In [2]:
df_applicant_experience = pd.read_csv('data/df_applicant_experience.csv', index_col=['ApplicantExperienceID'])
df_applicant_experience.drop(columns=['Unnamed: 0'], inplace=True)
df_applicant_experience.drop_duplicates(['ApplicantID', 'DateFrom', 'DateTo'], inplace=True)

df_applicant_experience.DateFrom = df_applicant_experience.DateFrom.fillna(str(0))
df_applicant_experience.DateTo = df_applicant_experience.DateTo.fillna(str(0))

In [3]:
df_applicant_experience.JobDescription.fillna(0, inplace=True)

In [4]:
df_applicant_experience[df_applicant_experience.select_dtypes(object).columns] = df_applicant_experience.select_dtypes(object).astype(str)

In [5]:
df_applicant_experience.head(3)

,ApplicantID,DateFrom,DateTo,Industry,CompanyName,JobDescription,Position,Salary
ApplicantExperienceID,,,,,,,,
1,3,2016-04-01 00:00:00.0000000,2017-04-01 00:00:00.0000000,Telekomunikasi,PT Icool International,<p>- Maintenance Dealer (cek stock dealer apak...,Sales Executive,3000000.0
2,3,2017-04-01 00:00:00.0000000,2017-07-01 00:00:00.0000000,Retail,PT DBL Indonesia,<p>- Melakukan kerjasama dengan pihak sekolah ...,Whole Sales,3800000.0
3,3,2017-08-01 00:00:00.0000000,2018-03-01 00:00:00.0000000,Service,PT Maybank Indonesia Finance,<p>- Melakukan Analisa kelayakan kredit mobil ...,Credit Marketing Officer,4200000.0


In [6]:
df_applicant_experience.DateFrom = df_applicant_experience.DateFrom.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
df_applicant_experience.DateTo = df_applicant_experience.DateTo.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))

df_applicant_experience.DateFrom = pd.to_datetime(df_applicant_experience.DateFrom)
df_applicant_experience.DateTo = pd.to_datetime(df_applicant_experience.DateTo)

# mengubah DateFrom dan DateTo menjadi sama-sama NaT jika salah satunya mengandung NaT
df_applicant_experience.DateFrom[
    (df_applicant_experience.DateFrom.isnull()) | (df_applicant_experience.DateTo.isnull())
] = pd.NaT
df_applicant_experience.DateTo[
    (df_applicant_experience.DateFrom.isnull()) | (df_applicant_experience.DateTo.isnull())
] = pd.NaT

C:\Users\febri\AppData\Local\Temp\ipykernel_8356\4048487225.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_applicant_experience.DateFrom[
C:\Users\febri\AppData\Local\Temp\ipykernel_8356\4048487225.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_applicant_experience.DateTo[


In [7]:
df_applicant_experience['YearsOfExperience'] = substract_months(
    df_applicant_experience.DateFrom, df_applicant_experience.DateTo
).apply(lambda x: int(x/12) if (x > 1) else 0)

In [8]:
df_applicant_experience.head(3)

,ApplicantID,DateFrom,DateTo,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience
ApplicantExperienceID,,,,,,,,,
1,3,2016-04-01,2017-04-01,Telekomunikasi,PT Icool International,<p>- Maintenance Dealer (cek stock dealer apak...,Sales Executive,3000000.0,1
2,3,2017-04-01,2017-07-01,Retail,PT DBL Indonesia,<p>- Melakukan kerjasama dengan pihak sekolah ...,Whole Sales,3800000.0,0
3,3,2017-08-01,2018-03-01,Service,PT Maybank Indonesia Finance,<p>- Melakukan Analisa kelayakan kredit mobil ...,Credit Marketing Officer,4200000.0,0


In [9]:
df_applicant_experience.Industry = df_applicant_experience.Industry.map(str.lower).apply(lambda x: re.sub('[^a-zA-Z\s]', '', x))
df_applicant_experience.CompanyName = df_applicant_experience.CompanyName.map(str.lower).apply(lambda x: re.sub('[^a-zA-Z\s]', '', x))

In [10]:
df_applicant_experience.head(3)

,ApplicantID,DateFrom,DateTo,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience
ApplicantExperienceID,,,,,,,,,
1,3,2016-04-01,2017-04-01,telekomunikasi,pt icool international,<p>- Maintenance Dealer (cek stock dealer apak...,Sales Executive,3000000.0,1
2,3,2017-04-01,2017-07-01,retail,pt dbl indonesia,<p>- Melakukan kerjasama dengan pihak sekolah ...,Whole Sales,3800000.0,0
3,3,2017-08-01,2018-03-01,service,pt maybank indonesia finance,<p>- Melakukan Analisa kelayakan kredit mobil ...,Credit Marketing Officer,4200000.0,0


In [11]:
df_applicant_experience.JobDescription = df_applicant_experience.JobDescription.map(str.lower).apply(
    lambda x: remove_morespace(
                remove_single(
                    maintain_alpha(
                        remove_html(
                            x
                        )
                    )
                )
            ).strip()
    )

c:\Users\febri\projects\eris_job\transform.py:46: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 46 of the file c:\Users\febri\projects\eris_job\transform.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  return BeautifulSoup(text).get_text()


In [12]:
df_applicant_experience.head(4)

,ApplicantID,DateFrom,DateTo,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience
ApplicantExperienceID,,,,,,,,,
1,3,2016-04-01,2017-04-01,telekomunikasi,pt icool international,maintenance dealer cek stock dealer apakah mas...,Sales Executive,3000000.0,1
2,3,2017-04-01,2017-07-01,retail,pt dbl indonesia,melakukan kerjasama dengan pihak sekolah untuk...,Whole Sales,3800000.0,0
3,3,2017-08-01,2018-03-01,service,pt maybank indonesia finance,melakukan analisa kelayakan kredit mobil konsu...,Credit Marketing Officer,4200000.0,0
4,10,NaT,NaT,perhotelan,country heritage hotel surabaya,bertugas sebagai receptionist sekaligus merang...,FRONT DESK AGENT - NIGHT AUDIT,3600000.0,0


In [13]:
df_applicant_experience.Position = df_applicant_experience.Position.map(str.lower).map(maintain_alpha).map(remove_morespace).map(str.strip)

In [14]:
df_applicant_experience.head(3)

,ApplicantID,DateFrom,DateTo,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience
ApplicantExperienceID,,,,,,,,,
1,3,2016-04-01,2017-04-01,telekomunikasi,pt icool international,maintenance dealer cek stock dealer apakah mas...,sales executive,3000000.0,1
2,3,2017-04-01,2017-07-01,retail,pt dbl indonesia,melakukan kerjasama dengan pihak sekolah untuk...,whole sales,3800000.0,0
3,3,2017-08-01,2018-03-01,service,pt maybank indonesia finance,melakukan analisa kelayakan kredit mobil konsu...,credit marketing officer,4200000.0,0


In [15]:
df_applicant_experience.Salary = df_applicant_experience.Salary.map(repair_salary)

In [16]:
df_applicant_experience.head(3)

,ApplicantID,DateFrom,DateTo,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience
ApplicantExperienceID,,,,,,,,,
1,3,2016-04-01,2017-04-01,telekomunikasi,pt icool international,maintenance dealer cek stock dealer apakah mas...,sales executive,3000000,1
2,3,2017-04-01,2017-07-01,retail,pt dbl indonesia,melakukan kerjasama dengan pihak sekolah untuk...,whole sales,3800000,0
3,3,2017-08-01,2018-03-01,service,pt maybank indonesia finance,melakukan analisa kelayakan kredit mobil konsu...,credit marketing officer,4200000,0


In [17]:
df_applicant_experience = df_applicant_experience.sort_values('DateFrom').groupby(['ApplicantID']).agg({
    'DateFrom': 'first',
    'DateTo': 'last',
    'Industry': ' '.join,
    'CompanyName': ' '.join,
    'JobDescription': ' '.join,
    'Position': ' '.join,
    'Salary': 'mean',
    'YearsOfExperience': 'sum',
})

df_applicant_experience.Salary = df_applicant_experience.Salary.astype(int)

In [18]:
df_applicant_experience.head(3)

,DateFrom,DateTo,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience
ApplicantID,,,,,,,,
1,2015-10-01,2019-02-01,jasa kawal angkut uang tunai architect consultant,pt wiratanu persada tama green building widya ...,pembuatan kontrak kerja penilaian kinerja kary...,staff hrd freelance drafter,3500000,4
3,2016-04-01,2018-03-01,telekomunikasi retail service,pt icool international pt dbl indonesia pt may...,maintenance dealer cek stock dealer apakah mas...,sales executive whole sales credit marketing o...,3666666,1
10,2014-02-01,2016-11-01,event organizer perhotelan,victory event organizer country heritage hotel...,bertugas sebagai marketing yang juga merangkap...,marketing administrasi front desk agent night ...,3200000,2


In [19]:
df_applicant_experience.drop(columns=['DateFrom', 'DateTo'], inplace=True)

In [20]:
df_applicant_experience.head(2)

,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience
ApplicantID,,,,,,
1,jasa kawal angkut uang tunai architect consultant,pt wiratanu persada tama green building widya ...,pembuatan kontrak kerja penilaian kinerja kary...,staff hrd freelance drafter,3500000,4
3,telekomunikasi retail service,pt icool international pt dbl indonesia pt may...,maintenance dealer cek stock dealer apakah mas...,sales executive whole sales credit marketing o...,3666666,1


In [24]:
df_applicant_experience.to_csv('data/cdf_applicant_experience.csv')